# Sales Forecasting cont. (pt.3)

### Modeling with the secondary objective of:
**Identify Key Sales Drivers:** 
- Uncover the factors that significantly impact sales. 
- By analyzing product categories, sales channels, promotional activities, and customer purchasing patterns, we can pinpoint what influences buyers. 
- This insight will inform targeted marketing and promotional strategies, boosting sales efficiency.

---
### <u>Retrieving Data from pt.1

- #### Installing Libraries

In [1]:
# INSTALLATIONS
# !pip install pandas
# !pip install openpyxl
# !pip install numpy
# !pip install xgboost
# !pip install lightgbm
# !pip install category_encoders


- #### Importing Libraries into Project
    - Setting pandas parameters to see the full amount of information from my dataset

In [2]:
import pandas as pd
import numpy as np
import os

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

- #### Reading file into my project

In [3]:
f_path = r'/content/drive/MyDrive/UCBerkeley/SalesForecasting/data/AmazonSalesReport.csv'
csv_path = r'data\AmazonSalesReport.csv'
xlsx_path = r'data\AmazonSalesReport.xlsx'

if os.path.isfile(csv_path):
    print('Reading File.')
else:
    print("Need to create this file")
    # Load the XLSX file
    # df = pd.read_excel(xlsx_path)
    # -----------------------------------------
    # Save to CSV format
    # df.to_csv(csv_path, index=False)  # Set index=False to not write row indices

df = pd.read_csv(csv_path)

Reading File.


- #### Grabbing a sample of my dataset to reference decisions from part 1 of the project

In [4]:
df.sample(1)

,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
61408,404-9540793-1177956,2022-05-22,Shipped,Amazon,Amazon.in,Expedited,SET293,SET293-KR-NP-S,Set,S,B09K3G4TS6,Shipped,1,INR,736.0,IMPHAL,MANIPUR,795001.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN,0.0


---
### <u>Addressing Duplicates

- #### Removing Duplicates with Order ID the Same
    - False-Positive Transactions

In [5]:
duplicates = df.duplicated().sum()
print("\nNumber of duplicate rows:", duplicates)
print(f"Number of rows Before Drop: {df.shape[0]}")
df = df.copy().drop_duplicates()
print(f"Number of rows After Drop: {df.shape[0]}")


Number of duplicate rows: 6
Number of rows Before Drop: 128975
Number of rows After Drop: 128969


---
### <u>Addressing Unnecessary Features

- #### Only Keeping Necessary Columns for Analysis

In [6]:
# Columns to keep
cols_needed = ['Category', 'Style', 'Size', 'SKU', 'Qty', 'currency', 'Amount', 'Sales Channel ', 'promotion-ids', 'B2B']
df = df[cols_needed]
df.columns

Index(['Category', 'Style', 'Size', 'SKU', 'Qty', 'currency', 'Amount',
       'Sales Channel ', 'promotion-ids', 'B2B'],
      dtype='object')

---
### <u>Addressing Missing Values

- #### Filling missing values in 'promotion-ids' with "No Promotion"

In [7]:
df['promotion-ids'].fillna("No Promotion", inplace=True)

- #### Drop missing values to disclude any potential biases
    - Since 'Amount' is crucial for analysis
    - There is only 1 type of currency wich is INR (Indian Rupee) * 0.012 for USD conversion

In [8]:
df.dropna(subset=['currency', 'Amount'], inplace=True)
df.drop(['currency', 'Sales Channel '], inplace=True, axis=1)

- #### Confirm changes by displaying remaining missing values, if any

In [9]:
df.isnull().sum().sort_values(ascending=False)

Category         0
Style            0
Size             0
SKU              0
Qty              0
Amount           0
promotion-ids    0
B2B              0
dtype: int64

---
### <u>Addressing Skewness

- #### Getting original skew of target variable

In [10]:
num_cols = df.select_dtypes(exclude='object').columns.to_list()
num_cols.remove('B2B')
for col in num_cols:
    print(f"Skew for {col}: {df[col].skew()}")

Skew for Qty: -2.689053634974344
Skew for Amount: 0.8855215498451299


- #### Introducing functions to handle skewness of dataset
    
    - transforming target variable column and present a more evenly distributed data to analyze
    - will need to convert back to original form later

In [11]:
# 
#
#
#
def cap_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[column] = df[column].clip(lower=lower_bound, upper=upper_bound)
    return df
#
#
#
#
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
#
#
#
#
def transform_feature(data, col):
    best_skew=0.5
    best_transformation = None
    transformed_name = None
    transformations = {
        "sqrt": np.sqrt(data[col]),
        "p1_3": data[col]**(1/3),
        "p1_4": data[col]**(1/4),
        "p1_5": data[col]**(1/5),
        "log": np.log(data[col] + 1e-10)
    }
    for k,v in transformations.items():
        if abs(v.skew()) < abs(best_skew):
            best_transformation = v
            transformed_name = f'{col}_{k}'
        else:
            continue
    if best_transformation != None:
        data[transformed_name] = best_transformation
        print('New feature transformed:',transformed_name)
        return data
    else:
        return data

- #### Implementing Dictionary for Functions to be Analyzed

In [12]:
decision_df = []
outlier_decisions = {
    "Original": df,
    "Remove": remove_outliers(df.copy(), 'Amount'),
    "Capping": cap_outliers(df.copy(), 'Amount'),
    "Transforming" : transform_feature(df.copy(), 'Amount')
}

- #### Putting Functions into a Table

In [13]:
for col in num_cols:
    for k,v in outlier_decisions.items():
        decision = v
        data_reduction_qty = 100 - (len(v) / len(df) * 100)
        decision_df.append({
            'Decision': k,
            'Feature': col, 
            'Data Reduction (Quantity)': data_reduction_qty,
            'Original Skew': df[col].skew(),
            'Target Feature Skew': decision[col].skew(),
            'Rows In Dataset': decision.shape[0],
        })
decision_df = pd.DataFrame(decision_df)
print("HANDLING OUTLIERS:")
decision_df

HANDLING OUTLIERS:


,Decision,Feature,Data Reduction (Quantity),Original Skew,Target Feature Skew,Rows In Dataset
0,Original,Qty,0.000000,-2.689054,-2.689054,121177
1,Remove,Qty,2.970861,-2.689054,-3.849247,117577
2,Capping,Qty,0.000000,-2.689054,-2.689054,121177
3,Transforming,Qty,0.000000,-2.689054,-2.689054,121177
4,Original,Amount,0.000000,0.885522,0.885522,121177
5,Remove,Amount,2.970861,0.885522,0.352470,117577
6,Capping,Amount,0.000000,0.885522,0.508087,121177
7,Transforming,Amount,0.000000,0.885522,0.885522,121177


- #### Deciding how I want to handle outliers(test_decisions)

In [14]:
df = outlier_decisions["Original"]
df.shape

(121177, 8)

---
### <u>Feature Engineering

- #### Convert boolean True/False feature "B2B" to 1/0

In [15]:
df['B2B'] = df['B2B'].astype(int)

In [16]:
df.columns

Index(['Category', 'Style', 'Size', 'SKU', 'Qty', 'Amount', 'promotion-ids',
       'B2B'],
      dtype='object')

In [17]:
df.isna().sum()

Category         0
Style            0
Size             0
SKU              0
Qty              0
Amount           0
promotion-ids    0
B2B              0
dtype: int64

---
### <u>Addressing High Cardinality Columns</u>
- Anything over 10 columns is considered high

- #### Preparing for Categorical Encoding

In [18]:
cat_cols = df.select_dtypes(include = 'object').columns.to_list()
cat_col_df = []
for k, unique in df[cat_cols].nunique().items():
    cat_col_df.append({
        'Name': k,
        'Unique Values': unique,
        'Dtype': df[k].dtype,
    })

pd.DataFrame(cat_col_df)

,Name,Unique Values,Dtype
0,Category,9,object
1,Style,1373,object
2,Size,11,object
3,SKU,7157,object
4,promotion-ids,5788,object


- #### Extracting only the important promotion details in the 'promotion-ids' column
    - This will check each cell; if it contains the specified phrase, it retains the promotion type only, otherwise, it leaves the cell as is.

In [19]:
df['promotion-ids'] = df['promotion-ids'].apply(lambda x: "Free-Financing" if "Free-Financing" in str(x) else x)
df['promotion-ids'] = df['promotion-ids'].apply(lambda x: "Free Shipping" if "Free Shipping" in str(x) else x)
df['promotion-ids'] = df['promotion-ids'].apply(lambda x: "Duplicated" if "Duplicated" in str(x) else x)
df['promotion-ids'] = df['promotion-ids'].apply(lambda x: "Coupon" if "Coupon" in str(x) else x)

df['promotion-ids'].value_counts().sort_values(ascending=False).head(10)

promotion-ids
Free Shipping     45856
No Promotion      41698
Free-Financing    32348
Duplicated          921
Coupon              354
Name: count, dtype: int64

- #### Extracting Size value "Free" and Changing it to "Any"
    - "Free" is not a size
    - Replacing "Free" with "Any"

In [20]:
print(f"Unique values in Size: {df['Size'].unique()}")
print(f"Occurances of 'Free' in 'Size': {df['Size'].value_counts().get('Free', 0)}")

df['Size'] = df['Size'].apply(lambda x: "Any" if "Free" in str(x) else x)
print("Size ready for Ordinal Encoding")

Unique values in Size: ['S' '3XL' 'XL' 'L' 'XXL' 'XS' '6XL' 'M' '4XL' '5XL' 'Free']
Occurances of 'Free' in 'Size': 356
Size ready for Ordinal Encoding


---
### <u>Splitting Dataset

- #### Train Test Split my Dataset

In [21]:
from sklearn.model_selection import train_test_split

X = df.drop(['Amount'], axis=1)
y = df['Amount']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
X_train.shape

(96941, 7)

### <u>Feature Encoding (Categorical)

- #### Separating High and Low Cardinality Columns

In [23]:
low_cardinality_cols = []
high_cardinality_cols = []
cat_cols = X_train.select_dtypes(include='object').columns.to_list()
for k,v in X_train[cat_cols].items():
    unique = len(X_train[k].unique())
    if unique < 15:
        low_cardinality_cols.append(k)
    else:
        high_cardinality_cols.append(k)
        
print(f"Low Cardinality Columns:\n{df[low_cardinality_cols].nunique()}")
print(f"\n\nHigh Cardinality Columns:\n{df[high_cardinality_cols].nunique()}")

Low Cardinality Columns:
Category          9
Size             11
promotion-ids     5
dtype: int64


High Cardinality Columns:
Style    1373
SKU      7157
dtype: int64


- #### One Hot Encoding on features with low cardinality
    - encoding features [placeholder] with One Hot Encoding

In [24]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='first')
ohe_features = low_cardinality_cols
ohe_features.remove('Size')

- #### Ordinal Encoding for "Size" column

In [25]:
from sklearn.preprocessing import OrdinalEncoder

categories = [['XS', 'S', 'M', 'L', 'XL', 'XXL', '3XL', '4XL', '5XL', '6XL', 'Any']]

ordinal_encoder = OrdinalEncoder(categories=categories)
ordinal_features = ['Size']

- #### Target Encoding on Features with High Cardinality

In [26]:
from category_encoders import TargetEncoder


target_features = high_cardinality_cols
target_encoder = TargetEncoder()

X_train[target_features] = target_encoder.fit_transform(X_train[target_features], y_train)

X_test[target_features] = target_encoder.transform(X_test[target_features])

### <u>Feature Scaling (Numeric)

- #### Separating Numeric Columns

In [27]:
num_cols = X_train.select_dtypes(exclude='object').columns.to_list()
print(num_cols)
for k,v in X_train[num_cols].items():
    print(f"Minimum value of {k}: {min(v)}")
    print(f"Maximum value of {k}: {max(v)}\n")

['Style', 'SKU', 'Qty', 'B2B']
Minimum value of Style: 286.8449096534543
Maximum value of Style: 1578.3547145599168

Minimum value of SKU: 297.9827797060718
Maximum value of SKU: 1452.988194507805

Minimum value of Qty: 0
Maximum value of Qty: 8

Minimum value of B2B: 0
Maximum value of B2B: 1



- #### Scaling Numerical Features
    - Scaling Numeric Features with a MinMax Scaler

In [28]:
from sklearn.preprocessing import MinMaxScaler

min_max_features = num_cols
min_max_scaler = MinMaxScaler()

### <u>Preprocessor Building

In [29]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', one_hot_encoder, ohe_features),
        ('ordinal', ordinal_encoder, ordinal_features),
        ('scaler', min_max_scaler, min_max_features)
    ]
)

- #### Fit and Transform the Preprocessor for Feature Selection/Modeling

In [30]:
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [31]:
X_train_processed.shape

(96941, 17)

In [32]:
new_feature_names = preprocessor.get_feature_names_out()
new_feature_names

array(['onehot__Category_Bottom', 'onehot__Category_Dupatta',
       'onehot__Category_Ethnic Dress', 'onehot__Category_Saree',
       'onehot__Category_Set', 'onehot__Category_Top',
       'onehot__Category_Western Dress', 'onehot__Category_kurta',
       'onehot__promotion-ids_Duplicated',
       'onehot__promotion-ids_Free Shipping',
       'onehot__promotion-ids_Free-Financing',
       'onehot__promotion-ids_No Promotion', 'ordinal__Size',
       'scaler__Style', 'scaler__SKU', 'scaler__Qty', 'scaler__B2B'],
      dtype=object)

---
### <u>Selecting Models to Use</u>
#### Goals:
- 
#### Explanation:
- 

In [33]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR


model_dict = {
    'RandomForest': RandomForestRegressor(),
    'XGBoost': XGBRegressor(),
    'LightGBM': LGBMRegressor(force_col_wise=True),
    'LinearRegression': LinearRegression(),
    'GradientBoosting': GradientBoostingRegressor(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),    
#     'SVR': SVR()
}

### <u>Cross Validation Scoring

In [34]:
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline

# Define the scoring metrics
scoring_metrics = {
    'mse': 'neg_mean_squared_error',
    'mae': 'neg_mean_absolute_error',
    'r2': 'r2'
}

# Initialize an empty list to store results
algo_score_dict = []

# Loop over each model in the model dictionary
for model_name, model in model_dict.items():
    # Create a pipeline with preprocessing and the current model
    pipe = Pipeline(steps=[
        ('preprocessing', preprocessor),
        ('model', model)
    ])
    
    # Use cross_validate to evaluate the model
    cv_results = cross_validate(pipe, 
                                X_train, 
                                y_train, 
                                cv=5, 
                                n_jobs=1,
                                verbose=0,
                                scoring=scoring_metrics,
                                return_train_score=False,
                                )
    
    # Calculate the mean of the scores and times for each metric
    for metric in scoring_metrics.keys():
        mean_score = np.mean(cv_results[f'test_{metric}'])
        fit_time = np.mean(cv_results['fit_time'])
        score_time = np.mean(cv_results['score_time'])
        
        # Append the results to the algo_score_dict list
        algo_score_dict.append({
            'Model Name': model_name,
            'Scoring Method': metric,
            'Mean CV Score': mean_score,
            'Mean Fit Time (seconds)': fit_time,
            'Mean Score Time (seconds)': score_time
        })
    print(f"\nFinished with {model_name}.")


Finished with RandomForest.

Finished with XGBoost.
[LightGBM] [Info] Total Bins 549
[LightGBM] [Info] Number of data points in the train set: 77552, number of used features: 16
[LightGBM] [Info] Start training from score 649.789985
[LightGBM] [Info] Total Bins 548
[LightGBM] [Info] Number of data points in the train set: 77553, number of used features: 16
[LightGBM] [Info] Start training from score 648.301711
[LightGBM] [Info] Total Bins 547
[LightGBM] [Info] Number of data points in the train set: 77553, number of used features: 16
[LightGBM] [Info] Start training from score 649.812126
[LightGBM] [Info] Total Bins 548
[LightGBM] [Info] Number of data points in the train set: 77553, number of used features: 16
[LightGBM] [Info] Start training from score 648.927033
[LightGBM] [Info] Total Bins 549
[LightGBM] [Info] Number of data points in the train set: 77553, number of used features: 16
[LightGBM] [Info] Start training from score 649.122472

Finished with LightGBM.

Finished with Li

In [35]:
results_df = pd.DataFrame(algo_score_dict)
results_df

,Model Name,Scoring Method,Mean CV Score,Mean Fit Time (seconds),Mean Score Time (seconds)
0,RandomForest,mse,-14493.279776,12.760801,0.337192
1,RandomForest,mae,-49.614554,12.760801,0.337192
2,RandomForest,r2,0.817260,12.760801,0.337192
3,XGBoost,mse,-13801.987699,0.195942,0.026808
4,XGBoost,mae,-52.004429,0.195942,0.026808
5,XGBoost,r2,0.825962,0.195942,0.026808
6,LightGBM,mse,-13942.970361,0.199890,0.027085
7,LightGBM,mae,-52.818296,0.199890,0.027085
8,LightGBM,r2,0.824200,0.199890,0.027085
9,LinearRegression,mse,-16514.424553,0.069108,0.017153


### <u>Grid Search with Cross Validation</u>

In [36]:
y_train.value_counts(normalize=True).mean()

0.0007407407407407406